## Coding Transformer from scratch in Pytorch

In [1]:
# Importing "lightning" to make it way easier to write our code  and for automatic code optimization and scaling in the cloud!
! pip3 install lightning


[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

import lightning as L

from position_encoding import PositionEncoding
from attention import Attention
from decoder_only_transformer import DecoderOnlyTransformer

### Prompts
    What is Statquest? Awesome

    Statquest is what? Awesome

In [3]:
token_to_id = {
    "what": 0,
    "is": 1,
    "statquest": 2,
    "awesome": 3,
    "<EOS>": 4
}

In [4]:
id_to_token = {id:token for token, id in token_to_id.items()}

In [5]:
inputs = torch.tensor([[token_to_id["what"],
                        token_to_id["is"],
                        token_to_id["statquest"],
                        token_to_id["<EOS>"],
                        token_to_id["awesome"]],

                        [token_to_id["statquest"],
                         token_to_id["is"],
                         token_to_id["what"],
                         token_to_id["<EOS>"],
                         token_to_id["awesome"]]])

labels = torch.tensor([[token_to_id["is"],
                        token_to_id["statquest"],
                        token_to_id["<EOS>"],
                        token_to_id["awesome"],
                        token_to_id["<EOS>"]],

                        [token_to_id["is"],
                        token_to_id["what"],
                        token_to_id["<EOS>"],
                        token_to_id["awesome"],
                        token_to_id["<EOS>"]]])

In [6]:
dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

In [8]:
# class PositionEncoding(nn.Module):
#     def __init__(self, d_model=2, max_len=6):
#         '''
#         d_model: dimention of the model, number of word embedding values per token
#         max_len: the maximum number of tokens our simpleLLm can process -- input and output combined
        
#         PE(pos,2i) = sin(pos/ 10000^(2i/model))
#         PE(pos,2i+1) = cos(pos/ 10000^(2i/model))
#         '''
#         super().__init__()

#         pe = torch.zeros(max_len, d_model)

#         position = torch.arange(start=0, end=max_len, step=1).float().unsqueeze(1)
#         # torch.arange() = to create a sequence of numbers
#         # unsqueeze(1) = turns the sequence to a column matrix
#         embedding_index = torch.arange(start=0, end=d_model, step=2).float()
#         # setting step=2 results in the same sequence numbers that we would get if we multiplied i by 2. So we save ourselves a little math!
        
#         div_term = 1/torch.tensor(10000)**(embedding_index / d_model)

#         pe[:, 0::2] = torch.sin(position * div_term)
#         pe[:, 1::2] = torch.cos(position * div_term)

#         self.register_buffer('pe', pe)
#         # To ensure that pe gets moved to a GPU if we use one!

#     def forward(self, word_embeddings):
#         return word_embeddings + self.pe[:word_embeddings.size(0), :]

In [9]:
# class Attention(nn.Module):
#     def __init__(self, d_model=2):
#         # d_model: number of word embedding values per token
#         super().__init__()
#         self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
#         self.W_k = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
#         self.W_v = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
#         # In the original Transformers manuscript, they don't add additional bias terms when calculating Attention, so we won't either by setting bias = False.
#         # As a result, we end up with an object we're calling W_q with the currently untrained Weights needed to calculate Query values.
       
#         self.row_dim = 0
#         self.col_dim = 1
#         # Just to give us flexibility to input training data in sequentially or in batches, we creat some variables to keep track of which indices are for rows and columns.

#     def forward(self, encodings_for_q, encodings_for_k, encodings_for_v, mask=None):

#         ''' We are doing for the sake of flexibility is allowing the Query, Key and Values 
#             to be calculated from different token encodings.
#         '''
        
#         q = self.W_q(encodings_for_q)
#         k = self.W_q(encodings_for_k)
#         v = self.W_q(encodings_for_v)

#         sims = torch.matmul(q, k.transpose(dim0=self.row_dim, dim1=self.col_dim))
#         scaled_sims = sims / torch.tensor(k.size(self.col_dim)**0.5)

#         if mask is not None:
#             scaled_sims = scaled_sims.masked_fill(mask=mask, value=-1e9)
#             # Masking is used to prevent early tokens from cheating and looking at later tokens.
        
#         attention_percents = F.softmax(scaled_sims, dim=self.col_dim)
#         attention_scores = torch.matmul(attention_percents, v)

#         return attention_scores


In [10]:
# class DecoderOnlyTransformer(L.LightningModule):
#     '''
#     Doing it this way, rather than having every class inherit from
#     LightningModule, allows us to take advantage of everything Lightning offers without the overhead of inheriting it multiple times.
#     '''
#     def __init__(self, num_tokens=4, d_model=2, max_len=6):
#         # max_len: Maximum lenght of the input + output
#         super().__init__()

#         self.we = nn.Embedding(num_embeddings= num_tokens, embedding_dim=d_model)
#         self.pe = PositionEncoding(d_model=d_model, max_len=max_len)
#         self.self_attention = Attention(d_model=d_model)
#         self.fc_layer = nn.Linear(in_features=d_model, out_features=num_tokens)

#         self.loss = nn.CrossEntropyLoss()

#     def forward(self, token_ids):
#         word_embeddings = self.we(token_ids)
#         position_encoded = self.pe(word_embeddings)

#         mask = torch.tril(torch.ones((token_ids.size(dim=0), token_ids.size(dim=0))))
#         # We create the mask that will prevent early tokens from looking at late tokens when we calculate Attention.
#         mask=mask==0

#         self_attention_values = self.self_attention(position_encoded,
#                                                     position_encoded,
#                                                     position_encoded,
#                                                     mask=mask)
        
#         residual_connection_values = position_encoded + self_attention_values
#         fc_layer_output = self.fc_layer(residual_connection_values)

#         return fc_layer_output

In [7]:
def configure_optimizers(self):
    return Adam(self.parameters(), lr=0.1)
    
def training_step(self, batch, batch_idx):
    input_tokens, labels = batch
    output = self.forward(input_tokens[0])
    loss = self.loss(output, labels[0]) 
    return loss

In [8]:
model = DecoderOnlyTransformer(num_tokens=len(token_to_id), d_model=2, max_len=6)

In [9]:
trainer = L.Trainer(max_epochs=30)
trainer.fit(model, train_dataloaders=dataloader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


MisconfigurationException: No `training_step()` method defined. Lightning `Trainer` expects as minimum a `training_step()`, `train_dataloader()` and `configure_optimizers()` to be defined.

In [10]:
model_input = torch.tensor([token_to_id["what"],
                            token_to_id["is"],
                            token_to_id["statquest"],
                            token_to_id["<EOS>"]])

input_length = model_input.size(dim=0)
predictions = model(model_input)
predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
print(predicted_id)
predicted_ids = predicted_id

max_length = 6
for i in range(input_length, max_length) :
    if (predicted_id == token_to_id["<EOS>"]):
        break
    
    model_input = torch.cat((model_input, predicted_id))

    predictions = model(model_input)
    predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
    predicted_ids = torch.cat((predicted_ids, predicted_id) )

print("Predicted Tokens:\n")
for id in predicted_ids:
    print("\t", id_to_token[id.item()])



tensor([0])
Predicted Tokens:

	 what
	 what
	 is
